# Normal Encoder of 24 bit

In [36]:
from PIL import Image, ImageDraw
from typing import Tuple

def create_color_logo_from_number(number: int, 
                               color0: Tuple[int, int, int]=(0, 0, 255), 
                               color1: Tuple[int, int, int]=(0, 0, 139)) -> Image.Image:


    if number < 0 or number >= (1 << 24):
        raise ValueError(f"Number must be between 0 and {(1 << 24) - 1}")

    binary_string = format(number, '024b')[-24:] 

    # Image setup
    img_size = 500
    symbol_radius = 20
    padding = 30
    symbols_per_row = 8
    spacing = (img_size - 2 * padding) // (symbols_per_row - 1)

    img = Image.new("RGB", (img_size, img_size), "white")
    draw = ImageDraw.Draw(img)

    for i, bit in enumerate(binary_string):
        row = i // symbols_per_row
        col = i % symbols_per_row
        x = padding + col * spacing
        y = padding + row * spacing
        color = color1 if bit == '1' else color0
        draw.ellipse([x - symbol_radius, y - symbol_radius, x + symbol_radius, y + symbol_radius], fill=color)

    return img



encoded_image = create_color_logo_from_number(11354783)
encoded_image.show()
encoded_image.save("encoded_logo.jpg", format="JPEG")


# Decoder of 24 bit 

In [35]:
from PIL import Image, ImageStat
from typing import Tuple, Dict

def decode_color_logo(image: Image.Image, 
                     color0: Tuple[int, int, int]=(0, 0, 255), 
                     color1: Tuple[int, int, int]=(0, 0, 139)) -> Tuple[str, int]:
    # Validate image
    if image.mode != 'RGB':
        raise ValueError("Image must be in RGB mode")

    img_size = image.size[0]
    if img_size != 500:  
        raise ValueError("Image size must be 500x500 pixels")

    symbol_radius = 20
    padding = 30
    symbols_per_row = 8
    spacing = (img_size - 2 * padding) // (symbols_per_row - 1)

    binary_string = ""

    for i in range(24):
        row = i // symbols_per_row
        col = i % symbols_per_row
        x = padding + col * spacing
        y = padding + row * spacing
        
        symbol_box = (
            x - symbol_radius, y - symbol_radius,
            x + symbol_radius, y + symbol_radius
        )
        try:
            symbol_region = image.crop(symbol_box)
            # Get average color
            stat = ImageStat.Stat(symbol_region)
            avg_color = tuple(int(c) for c in stat.mean[:3])
            
            # Compare distances to color0 and color1
            dist0 = sum((a - b) ** 2 for a, b in zip(avg_color, color0))
            dist1 = sum((a - b) ** 2 for a, b in zip(avg_color, color1))
            
            binary_string += '0' if dist0 < dist1 else '1'
        except Exception as e:
            raise ValueError(f"Error processing symbol at position {i}: {str(e)}")

    try:
        # Convert binary string to number
        decoded_number = int(binary_string, 2)
        return binary_string, decoded_number
    except ValueError as e:
        raise ValueError(f"Error converting binary string to number: {str(e)}")



binary_result, decoded_number = decode_color_logo(encoded_image)
binary_result, decoded_number


('101011010100001010011111', 11354783)

# SBI LOGO

In [37]:
from PIL import Image, ImageDraw

def generate_sbi_logo(image_size=512):
    # Colors
    background_color = (255, 255, 255)
    logo_blue = (0, 102, 204)  # SBI blue

    # Create canvas
    img = Image.new("RGB", (image_size, image_size), background_color)
    draw = ImageDraw.Draw(img)

    center = image_size // 2
    outer_radius = center - 10
    inner_circle_radius = outer_radius // 6
    slit_width = image_size // 20
    slit_height = center - inner_circle_radius

    # Draw outer blue circle
    draw.ellipse(
        [center - outer_radius, center - outer_radius,
         center + outer_radius, center + outer_radius],
        fill=logo_blue
    )

    # Draw vertical white slit (rectangle)
    draw.rectangle(
        [center - slit_width // 2, center,
         center + slit_width // 2, center + slit_height],
        fill="white"
    )

    # Draw central white circle (keyhole top)
    draw.ellipse(
        [center - inner_circle_radius, center - inner_circle_radius,
         center + inner_circle_radius, center + inner_circle_radius],
        fill="white"
    )

    return img

# Generate and show the logo
logo = generate_sbi_logo()
logo.show()


# Encoded SBI LOGO

In [38]:
from PIL import Image, ImageDraw

def generate_sbi_logo(image_size=40, fill_color=(0, 102, 204)):
    # Colors
    background_color = (255, 255, 255)

    # Create canvas
    img = Image.new("RGBA", (image_size, image_size), (0,0,0,0))  # transparent background
    draw = ImageDraw.Draw(img)

    center = image_size // 2
    outer_radius = center - 2
    inner_circle_radius = outer_radius // 6
    slit_width = image_size // 10
    slit_height = center - inner_circle_radius

    # Draw outer circle
    draw.ellipse(
        [center - outer_radius, center - outer_radius,
         center + outer_radius, center + outer_radius],
        fill=fill_color
    )

    # Draw vertical white slit (rectangle)
    draw.rectangle(
        [center - slit_width // 2, center,
         center + slit_width // 2, center + slit_height],
        fill="white"
    )

    # Draw central white circle (keyhole top)
    draw.ellipse(
        [center - inner_circle_radius, center - inner_circle_radius,
         center + inner_circle_radius, center + inner_circle_radius],
        fill="white"
    )

    return img

def create_color_logo_from_number(number: int, 
                               color0=(0, 102, 204), 
                               color1=(204, 0, 0),
                               symbol_size=40,
                               image_size=500):
    if number < 0 or number >= (1 << 24):
        raise ValueError(f"Number must be between 0 and {(1 << 24) - 1}")

    binary_string = format(number, '024b')[-24:]

    symbols_per_row = 8
    padding = 30
    spacing = (image_size - 2 * padding - symbol_size) // (symbols_per_row - 1)

    img = Image.new("RGB", (image_size, image_size), "white")

    for i, bit in enumerate(binary_string):
        row = i // symbols_per_row
        col = i % symbols_per_row
        x = padding + col * spacing
        y = padding + row * spacing

        fill_color = color1 if bit == '1' else color0

        # Generate SBI logo with correct color and size
        symbol_img = generate_sbi_logo(image_size=symbol_size, fill_color=fill_color)

        # Paste with transparency support
        img.paste(symbol_img, (x, y), symbol_img)

    return img

# Example usage
encoded_image2 = create_color_logo_from_number(11568962)
encoded_image2.show()
encoded_image2.save("encoded_logo2.jpg", format="JPEG")



# Decoded SBI logo

In [39]:
from PIL import Image, ImageStat
import numpy as np

def decode_color_logo(image: Image.Image, 
                     color0=(0, 102, 204), 
                     color1=(204, 0, 0),
                     symbol_size=40,
                     image_size=500) -> Tuple[str, int]:
    
    if image.mode != 'RGB':
        raise ValueError("Image must be in RGB mode")

    if image.size != (image_size, image_size):
        raise ValueError(f"Image size must be {image_size}x{image_size} pixels")

    symbols_per_row = 8
    padding = 30
    spacing = (image_size - 2 * padding - symbol_size) // (symbols_per_row - 1)

    binary_string = ""

    for i in range(24):
        row = i // symbols_per_row
        col = i % symbols_per_row
        x = padding + col * spacing
        y = padding + row * spacing

        # Crop symbol area
        symbol_box = (x, y, x + symbol_size, y + symbol_size)
        symbol_region = image.crop(symbol_box)

        # Convert cropped symbol to numpy array
        arr = np.array(symbol_region)

        # Identify pixels that are NOT white (threshold)
        # White pixels have RGB close to (255,255,255)
        non_white_mask = np.any(arr[:, :, :3] < 240, axis=2)  # boolean mask of colored pixels

        # If no colored pixels found, assume bit 0 (safe fallback)
        if np.sum(non_white_mask) == 0:
            bit = '0'
        else:
            # Average color of non-white pixels
            avg_color = arr[:, :, :3][non_white_mask].mean(axis=0)

            # Calculate squared Euclidean distances to color0 and color1
            dist0 = np.sum((avg_color - np.array(color0))**2)
            dist1 = np.sum((avg_color - np.array(color1))**2)

            bit = '0' if dist0 < dist1 else '1'

        binary_string += bit

    decoded_number = int(binary_string, 2)
    return binary_string, decoded_number




binary_result, decoded_number = decode_color_logo(encoded_image2)
binary_result, decoded_number


('101100001000011101000010', 11568962)